In [1]:
import pandas as pd
import time
from datetime import datetime, timedelta
import json
import requests
import re
import numpy as np
import matplotlib as plt
from bs4 import BeautifulSoup

In [2]:
salaryHtml = requests.get('https://www.spotrac.com/nfl/contracts/cumulative-cash//')
bs = BeautifulSoup(salaryHtml.content, 'html.parser')
time.sleep(1)

#links for players
playersRaw = bs.find_all("td",{"class":"player"})
playersLinks = []
for td in playersRaw:
    playersLinks.append(td.find("a")["href"])

updatedPlayersLinks = playersLinks[:len(playersLinks) - 300]

In [3]:
PlayerList = []
PositionList = []
AgeList = []
TeamList = []
ExperienceList = [] 
DraftedDetailsList = []
CollegeList = []
AgentList = []
SalaryList = []
PlayerStatusList = []

PlayerDetailesRaw1 = []
PlayerDetailesRaw2 = []


In [4]:
for href in updatedPlayersLinks:
    CashearningHtml = requests.get(href + "cash-earnings/")
    bs = BeautifulSoup(CashearningHtml.content, 'html.parser')
    
    PlayerDetailesRaw1 = bs.find("div",{"class":"player-option medium"})
    PlayerDetailesRaw2 = bs.find("div",{"class":"player-option large"})
    
    #Player name
    try:
        PlayerList.append(bs.find("h1").text.strip())
    except:
        PlayerList.append(np.nan)
    
    #player position and team
    try:
        PlayerPos = PlayerDetailesRaw1.find("span",{"class":"player-item position"}).text.split(", ")
        TeamList.append(PlayerPos[0])
        PositionList.append(PlayerPos[1])
    except:
        TeamList.append(np.nan)
        PositionList.append(np.nan)
    
    #player age and experience  
    try:
        PlayerAgeExp = PlayerDetailesRaw1.find("span",{"class":"player-infoitem"})
        #searching text if exist
        if PlayerDetailesRaw1.find(text="Age:"):
            AgeList.append(PlayerAgeExp.text)
            PlayerAgeExp = PlayerAgeExp.findNext("span",{"class":"player-infoitem"})
        else:
            AgeList.append(np.nan)
        #searching text if exist
        if PlayerDetailesRaw1.find(text="Exp: "):
            ExperienceList.append(PlayerAgeExp.text)
        else:
            ExperienceList.append(np.nan)
    except:
        AgeList.append(np.nan)
        ExperienceList.append(np.nan)
    
     #player drafte details, college and agent  
    try:
        Player = PlayerDetailesRaw2.find("span",{"class":"player-infoitem"})
        if PlayerDetailesRaw2.find(text="Drafted:"):
            DraftedDetailsList.append(Player.text)
            Player = Player.findNext("span",{"class":"player-infoitem"})
        else:
            DraftedDetailsList.append(np.nan)
    
        if PlayerDetailesRaw2.find(text="College:"):
            CollegeList.append(Player.text)
            Player = Player.findNext("span",{"class":"player-infoitem"})
        else:
            CollegeList.append(np.nan)

        if PlayerDetailesRaw2.find(text="Agent(s):"):
            AgentList.append(Player.text)
        else:
            AgentList.append(np.nan)

    except:
        DraftedDetailsList.append(np.nan)
        CollegeList.append(np.nan)
        AgentList.append(np.nan)

    #player salary
    earnings = bs.find("table",{"class":"earningstable rtable"}).find("tbody").findAll("td",{"class":"right visible-lg"})
    if earnings:
        SalaryList.append(earnings[0].text)
    else:
        SalaryList.append(np.nan)
    
    #player status
    try:
        PlayerStatusList.append(bs.find("div",{"class":"player-status"}).text.strip())
    except:
        PlayerStatusList.append(np.nan)
        
    time.sleep(1)
    
df = pd.DataFrame({"Player Name":PlayerList,"Salary":SalaryList,"Experience Years":ExperienceList,"Team":TeamList,"Position":PositionList,"Age":AgeList,"Drafted Details":DraftedDetailsList,"College":CollegeList,"Agent":AgentList,"Player Status":PlayerStatusList})
df

KeyboardInterrupt: 

In [ ]:
df_try.to_csv("NFLPlayerDetails.csv",index= False)

In [ ]:
df_try = pd.read_csv("NFLPlayerDetails.csv")
df_try